# Rule Based System applied to Tarot

### Imports

In [31]:
import random

## Game Modelisation

Are needed :
- trump cards + the fool + oudlers
- figures = King, Queen, Paladin, Jack
- whiteskins = 1 to 10
- the dog (neither first 3 nor last but not important here)
- colors = hearts, clubs, diamonds and spades
- Hand
- taker


Steps :
- Dealing
- Biding
- Handful
- Play of the card

## Cards Definition

In [338]:
FIGURES = ['J','P','Q','K']
FIGURES_NAMES = ["Jack","Knight","Queen","King"]
NON_FIGURES = ['1','2','3','4','5','6','7','8','9','10']
CARD_VALUES = non_figures + figures

COLOR_VALUES = [ 'H', 'C', 'D', 'S']
COLOR_NAMES = ["Hearts","Clubs","Diamonds","Spades"]

TRUMPS = [ i for i in range(1,22)] + ['F']
OUDLERS = [1, 21, 'F']

COLOR_CARDS = [ COLOR_VALUES[j]+str(CARD_VALUES[i]) for j in range(4) for i in range(len(CARD_VALUES))]

PLAYING_CARDS = TRUMPS + COLOR_CARDS
PLAYING_CARDS

18

## Cards Recognition

In [236]:
def get_card_value(card):
    # TODO : check else clause validity
    return card[1:] if card not in TRUMPS else card

def is_figure(card):
    return get_card_value(card) in FIGURES

def is_non_figure(card):
    return get_card_value(card) in NON_FIGURES

def is_trump(card):
    return type(card) is int or card is 'F'

def is_oudler(card):
    return card in OULDERS

### TESTS ###
get_card_value(PLAYING_CARDS[77])
is_trump(PLAYING_CARDS[22])
mylist=[1 for card in playing_cards if is_trump(card)]
print(sum(mylist))

for i in range(len(PLAYING_CARDS)):
    to_string(PLAYING_CARDS[i])

22


## Dealing

In [243]:
def deal():
    '''
    Returns 5 args : 4 hands of 18 cards + dog
    '''
    deck = PLAYING_CARDS.copy()
    random.shuffle(deck)
    return [deck[0:18],deck[18:36],deck[36:54],deck[54:72],deck[72:78]]

## Hand Management : Sorting, Showing, Information

In [244]:
def get_index(card):
    return PLAYING_CARDS.index(card)

def sort_cards(hand):
    return hand.sort(key=get_index)

def to_string(card):
    if is_trump(card):
        if card is not 'F':
            return(f"Trump {card}")
        else:
            return(f"Trump Fool")
    else:
        return(f"{COLOR_NAMES[COLOR_VALUES.index(card[0])]} {FIGURES_NAMES[FIGURES.index(card[1:])] if is_figure(card) else card[1:]}")

def show_cards(cards):
    for card in cards:
        print(to_string(card))

def get_trump_count(hand):
    return sum([1 for card in hand if is_trump(card)])

### TESTS ###
for i in range(4):
    print(f"=== Showing hand {i+1} ===")
    sort_cards(hands[i])
    show_cards(hands[i])

=== Showing hand 1 ===
Trump 4
Trump 9
Trump 14
Trump 17
Trump 20
Hearts 3
Hearts 5
Hearts 8
Hearts 9
Hearts Jack
Clubs 4
Clubs 5
Diamonds 3
Diamonds 5
Diamonds 8
Spades 5
Spades 10
Spades King
=== Showing hand 2 ===
Trump 1
Trump 7
Trump 10
Trump 13
Trump 15
Trump 21
Hearts 6
Hearts Queen
Hearts King
Clubs 3
Clubs 10
Clubs Queen
Diamonds 1
Diamonds 2
Diamonds 7
Diamonds 10
Spades 8
Spades 9
=== Showing hand 3 ===
Trump 6
Trump 12
Trump 16
Trump 18
Hearts 1
Hearts 2
Hearts 4
Hearts Knight
Clubs 1
Clubs 2
Clubs 7
Diamonds Jack
Diamonds Knight
Spades 1
Spades 2
Spades 3
Spades 7
Spades Knight
=== Showing hand 4 ===
Trump 2
Trump 8
Trump 11
Trump 19
Trump Fool
Hearts 7
Hearts 10
Clubs 6
Clubs 8
Clubs 9
Clubs Jack
Clubs Knight
Diamonds 6
Diamonds 9
Diamonds King
Spades 4
Spades 6
Spades Jack


## Biding

In [376]:
CONTRACTS = [56,51,41,36]
POINTS = [1.5,2.5,3.5,4.5]

def get_card_points(card):
    if is_trump(card):        
        return 4.5 if card in OUDLERS else .5
    else:
        return .5 if card[1] in NON_FIGURES else POINTS[FIGURES.index(card[1])]

def compute_score(cards):
    return sum(get_card_points(card) for card in cards)

def get_contract(hand):
    return CONTRACTS[sum([1 for card in hand if is_oudler(card)])]

def estimate_win_factor(hand):
    '''
    2 essential params : number of trump cards + actual score
    each has .5 potential influence -> the closest to 1, the higher the chances of victory
    '''
    # compare actual points vs needed
    # TODO : avoid taking with score > 30 points gap (negative)
    score_param = .5 * (1 + (compute_score(hand) + 10 - get_contract(hand)) / get_contract(hand))
    
    # ratio of actual trump count VS mean    
    trump_mean = 0 if get_trump_count(hand)==0 else sum([10 if card is 'F' else card for card in hand if is_trump(card)]) / get_trump_count(hand)
    # 2 factors : nb_trumps & mean_trumps
    # nb_trumps_factor = 1 + (get_trump_count(hand)-(22/4))/10
    nb_trumps_factor = 1 + (get_trump_count(hand)/(22/4))/4
    mean_trumps_factor = 1 + (trump_mean-10) / 20
    trump_param = .5 * nb_trumps_factor * mean_trumps_factor

    if score_param + trump_param > 1:
        print(f"score_param : {score_param}, trump_param : {trump_param}\nnb_trumps_factor : {nb_trumps_factor}, mean_trumps_factor : {mean_trumps_factor}")
    return (score_param + trump_param)


### TESTS ###
# for card in PLAYING_CARDS:
#     print(f"{to_string(card)} : {get_card_points(card)}")

hand1,hand2,hand3,hand4,dog=deal()
hands = [hand1,hand2,hand3,hand4]

for i in range(4):
    print(f"hand {i} has contract {get_contract(hands[i])} with win factor {estimate_win_factor(hands[i])}")
    print(f"Actual score is {compute_score(hands[i])} and trump count is {get_trump_count(hands[i])}\n")

for i in range(4):
    if estimate_win_factor(hands[i]) > 1:
        print(f"=== Player {i} hand ===")
        sort_cards(hands[i])
        show_cards(hands[i])

score_param : 0.28431372549019607, trump_param : 0.8590909090909091
nb_trumps_factor : 1.2727272727272727, mean_trumps_factor : 1.35
hand 0 has contract 51 with win factor 1.1434046345811053
Actual score is 19.0 and trump count is 6

hand 1 has contract 51 with win factor 0.8599331550802138
Actual score is 17.0 and trump count is 5

hand 2 has contract 51 with win factor 0.8972482174688058
Actual score is 22.0 and trump count is 4

hand 3 has contract 56 with win factor 0.9317207792207791
Actual score is 27.0 and trump count is 5

score_param : 0.28431372549019607, trump_param : 0.8590909090909091
nb_trumps_factor : 1.2727272727272727, mean_trumps_factor : 1.35
=== Player 0 hand ===
Trump 13
Trump 15
Trump 16
Trump 17
Trump 20
Trump 21
Hearts 4
Hearts 8
Hearts 9
Hearts 10
Clubs 8
Clubs Knight
Diamonds 4
Diamonds 5
Diamonds Queen
Spades 3
Spades 8
Spades Jack


In [327]:
sort_cards(hands[3])
show_cards(hands[3])

Trump 1
Trump 2
Trump 5
Trump 6
Trump 15
Trump 17
Trump 21
Hearts 7
Hearts 9
Hearts 10
Hearts Knight
Clubs 9
Diamonds 8
Diamonds 10
Spades 3
Spades 7
Spades 10
Spades Queen


In [371]:
good_hand_1 = [1,2,3,6,7,8,9,21, 'HJ','HP','HQ','HK','DJ','DP','DQ','DK','CK','SK']

print(f"=== Player with good hand 1 hand ===")
print(f"good hand 1 has contract {get_contract(good_hand_1)} with win factor {estimate_win_factor(good_hand_1)}")
print(f"Actual score is {compute_score(good_hand_1)} and trump count is {get_trump_count(good_hand_1)}\n")
sort_cards(good_hand_1)
show_cards(good_hand_1)

=== Player with good hand 1 hand ===
score : 0.6707317073170732, trump : 0.5838068181818182
nb_trumps_factor : 1.3636363636363638, trump mean : 7.125, mean_trumps_factor : 0.85625
good hand 1 has contract 41 with win factor 1.2545385254988914
Actual score is 45.0 and trump count is 8

Trump 1
Trump 2
Trump 3
Trump 6
Trump 7
Trump 8
Trump 9
Trump 21
Hearts Jack
Hearts Knight
Hearts Queen
Hearts King
Clubs King
Diamonds Jack
Diamonds Knight
Diamonds Queen
Diamonds King
Spades King


In [370]:
good_hand_2 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

print(f"=== Player with good hand 2 hand ===")
print(f"good hand 2 has contract {get_contract(good_hand_2)} with win factor {estimate_win_factor(good_hand_2)}")
print(f"Actual score is {compute_score(good_hand_2)} and trump count is {get_trump_count(good_hand_2)}\n")
sort_cards(good_hand_2)
show_cards(good_hand_2)

=== Player with good hand 2 hand ===
score : 0.22549019607843135, trump : 0.8863636363636365
nb_trumps_factor : 1.8181818181818183, trump mean : 9.5, mean_trumps_factor : 0.975
good hand 2 has contract 51 with win factor 1.1118538324420677
Actual score is 13.0 and trump count is 18

Trump 1
Trump 2
Trump 3
Trump 4
Trump 5
Trump 6
Trump 7
Trump 8
Trump 9
Trump 10
Trump 11
Trump 12
Trump 13
Trump 14
Trump 15
Trump 16
Trump 17
Trump 18
